In [107]:
%load_ext autoreload
%autoreload 2

from Satellite import Satellite
from SatelliteState import SatelliteState
from Earth import Earth

import numpy as np

import matplotlib.pyplot


init_r = 6378136.6+200000
angular_vel = 0.001454
tang_vel = angular_vel * init_r
tang_vel = 60 * 4.848e-6 * init_r

sat_state1 = SatelliteState(np.array([init_r, np.pi/2, 0]), np.array([0]), np.array([0, tang_vel, 0]), np.array([0]))

earth = Earth()


sat1 = Satellite(sat_state1, 0, earth=earth)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
tang_vel

1913.4483742079997

In [109]:
sat1.true_state

SatelliteState(pos=array([6.57813660e+06, 1.57079633e+00, 0.00000000e+00]), cov=array([0]), velocity=array([   0.        , 1913.44837421,    0.        ]), acceleration=array([0]))

In [110]:
sat1.true_state.get_state_sat_plane()

array([6.57813660e+06, 0.00000000e+00, 1.57079633e+00, 1.91344837e+03])

In [111]:
sat1.d_state(0, sat1.true_state.get_state_sat_plane())

array([ 0.00000000e+00, -8.65499960e+00,  2.90880000e-04, -2.46467629e-06])

In [112]:
sat1.plane_to_altitude(init_r, np.pi/2)['distance']

221384.69999999925

In [113]:
sol = sat1.simulate(10000000)

In [114]:
sol

  message: Required step size is less than spacing between numbers.
  success: False
   status: -1
        t: [ 0.000e+00  1.000e+00 ...  9.470e+02  9.480e+02]
        y: [[ 6.578e+06  6.578e+06 ...  1.804e+05  1.064e+05]
            [ 0.000e+00 -8.655e+00 ... -6.562e+04 -8.589e+04]
            [ 1.571e+00  1.571e+00 ...  1.634e+00  1.634e+00]
            [ 1.913e+03  1.913e+03 ...  5.301e-01  3.127e-01]]
      sol: None
 t_events: [array([], dtype=float64)]
 y_events: [array([], dtype=float64)]
     nfev: 734
     njev: 0
      nlu: 0

In [116]:
sol.y[0]

array([6578136.6       , 6578132.27249829, 6578119.28998069,
       6578097.6524097 , 6578067.35972288, 6578028.41183289,
       6577980.80862738, 6577924.54996901, 6577859.63569545,
       6577786.06561938, 6577703.83952846, 6577612.95718537,
       6577513.4183278 , 6577405.22266843, 6577288.36989496,
       6577162.85967006, 6577028.69163145, 6576885.86539183,
       6576734.38053889, 6576574.23663534, 6576405.43321891,
       6576227.9698023 , 6576041.84587324, 6575847.06089277,
       6575643.61429548, 6575431.50549515, 6575210.7338804 ,
       6574981.29881459, 6574743.19963578, 6574496.43565675,
       6574241.00616499, 6573976.91042271, 6573704.14766683,
       6573422.71710898, 6573132.6179355 , 6572833.84930746,
       6572526.41036063, 6572210.30020549, 6571885.51792725,
       6571552.06258582, 6571209.93321582, 6570859.1288266 ,
       6570499.64840221, 6570131.49090142, 6569754.65525772,
       6569369.14037928, 6568974.94514903, 6568572.06842459,
       6568160.50903829,

In [117]:
# Plotting the results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(sol.t, sol.y[0], label='r(t)')
plt.title('r(t) over time')
plt.xlabel('Time')
plt.ylabel('r')
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(sol.t, sol.y[1], label='v_r(t)')
plt.title('v_r(t) over time')
plt.xlabel('Time')
plt.ylabel('v_r')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined